In [63]:
# Importing libraries
import pandas as pd
import csv
import json
from PIL import Image
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torchviz

In [64]:
# Paths for train and test images
raw_path = '/home/user/disk2/mary/xai606/raw_data/'
data_path = '/home/user/disk2/mary/xai606/final_dataset/'
train_path = f"{data_path}train_images"
test_path = f"{data_path}test_images"
model_path = './models/basemodel.pt'
effmodel_path = './models/effmodel.pt'
label_map = 'label_num_to_disease_map.json'

In [65]:
with open(raw_path+label_map, encoding='utf-8') as f:
    label_info = json.load(f)
print(json.dumps(label_info, indent=1))

{
 "0": "Cassava Bacterial Blight (CBB)",
 "1": "Cassava Brown Streak Disease (CBSD)",
 "2": "Cassava Green Mottle (CGM)",
 "3": "Cassava Mosaic Disease (CMD)",
 "4": "Healthy"
}


In [66]:
# Experiment setting
NUM_CLS = 5
BATCH = 32  # Batch size
EPOCHS = 20  # total number of epochs
LR = 1e-3  # learning rate
IM_SIZE = 256  # image size

# Device setting
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1, 2, 3, 4"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [67]:
# Method for getting data
class GetData(Dataset):
    def __init__(self, Dir, FNames, Labels, Transform):
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.lbs = Labels

    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):
        x = Image.open(os.path.join(self.dir, self.fnames[index]))
        if "train" in self.dir:
            return self.transform(x), self.lbs[index]
        elif "test" in self.dir:
            return self.transform(x), self.lbs[index]

In [68]:
# Train and test data and corresponding Labels for train images
labels = json.load(open(f"{raw_path}{label_map}"))
train = pd.read_csv(f'{data_path}/train.csv')
test = pd.read_csv(f'{data_path}/test.csv')
X_train, y_train = train['image_id'].values, train['label'].values
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, 
                                                  random_state=2022011107, shuffle=True, stratify=y_train)
X_test, y_test = test['image_id'].values, test['label'].values
print(X_train.shape, X_val.shape, X_test.shape)


(12838,) (4280,) (4279,)


In [69]:
# Data loading
Transform = transforms.Compose(
    [transforms.Resize((IM_SIZE, IM_SIZE)),
     transforms.RandomRotation(90),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

testset = GetData(test_path, X_test, y_test, Transform)
testloader = DataLoader(testset, batch_size=BATCH, shuffle=False, num_workers=2)

In [71]:
basemodel = torch.load(model_path)
basemodel.eval()

test_loss = 0.0
test_acc = 0.0
test_loss_l = []
test_acc_l = []

for i, (images, labels) in enumerate(tqdm(testloader)):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # att_map, logits = model(images)
    logits = basemodel(images)
    test_loss = criterion(logits, labels)

    test_loss += test_loss.detach().item()
    _, predicted = torch.max(logits, 1)
    test_acc += (predicted == labels).sum().item()
print('Epoch: %d | Test Loss: %.4f' % (epoch+1, test_loss / i), end="\t")
print('Epoch: %d | Test Acc: %.4f' % (epoch+1, test_acc / len(testset)))
test_acc_l.append(test_acc / len(testset))
test_loss_l.append(test_loss / i)

  0%|          | 0/134 [00:00<?, ?it/s]


RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.